In [3]:
import requests

# Constants
API_KEY = "NFtDR0vf_a82vRcKZwLtagApGCNJpm4X"
BASE_URL = "https://api.polygon.io/v3/snapshot/options/MSFT"

# Query Parameters
params = {
    "apiKey": API_KEY
}

# Send request to Polygon.io snapshot endpoint for full options chain
response = requests.get(BASE_URL, params=params)

# Raise error if failed
if response.status_code != 200:
    raise Exception(f"Failed to fetch option chain: {response.status_code} - {response.text}")

# Parse JSON response
option_chain_data = response.json()

# Display top-level keys and preview of results
option_chain_data.keys(), option_chain_data.get("results", [])[:5]  # Show first 2 contracts as sample


(dict_keys(['results', 'status', 'request_id', 'next_url']),
 [{'day': {'change': 0,
    'change_percent': 0,
    'close': 262.16,
    'high': 262.16,
    'last_updated': 1751486400000000000,
    'low': 262.16,
    'open': 262.16,
    'previous_close': 262.16,
    'volume': 1,
    'vwap': 262.16},
   'details': {'contract_type': 'call',
    'exercise_style': 'american',
    'expiration_date': '2025-07-11',
    'shares_per_contract': 100,
    'strike_price': 230,
    'ticker': 'O:MSFT250711C00230000'},
   'greeks': {'delta': 0.996946049112377,
    'gamma': 5.989022884704959e-05,
    'theta': -0.5398986229443306,
    'vega': 0.004453290013359785},
   'implied_volatility': 5.0533437746605605,
   'open_interest': 1,
   'underlying_asset': {'ticker': 'MSFT'}},
  {'day': {'change': 0,
    'change_percent': 0,
    'close': 256.8,
    'high': 258.34,
    'last_updated': 1751918400000000000,
    'low': 256.8,
    'open': 258.34,
    'previous_close': 256.8,
    'volume': 3,
    'vwap': 257.6033

In [2]:
!pip install scipy

  Using cached scipy-1.13.1-cp39-cp39-win_amd64.whl.metadata (60 kB)
Using cached scipy-1.13.1-cp39-cp39-win_amd64.whl (46.2 MB)


In [8]:
import requests
import pandas as pd
import numpy as np
from scipy.stats import norm
from datetime import datetime, timedelta, timezone
import logging

# --- Configuration ---
API_KEY = "67ffece4b2ae08.94077168"
BASE_URL = "https://eodhd.com/api/mp/unicornbay/options/contracts"
REAL_TIME_URL = "https://eodhd.com/api/real-time/{ticker}.US?api_token={api_key}&fmt=json"

logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s")

# --- Black-Scholes Pricing Functions ---

def black_scholes_call_price(S, K, T, r, sigma):
    if T <= 0 or sigma <= 0 or S <= 0 or K <= 0:
        return max(S - K, 0)
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

def black_scholes_put_price(S, K, T, r, sigma):
    if T <= 0 or sigma <= 0 or S <= 0 or K <= 0:
        return max(K - S, 0)
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)

def current_stock_price(ticker):
    url = REAL_TIME_URL.format(ticker=ticker, api_key=API_KEY)
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        data = response.json()
        return float(data.get("close", 0))
    except Exception as e:
        print(f"🚨 FAILED TO FETCH PRICE for {ticker}. Reason: {e}")
        return 0

# --- Main Simulation Function ---

def simulate_option_watchlist_single(ticker, option_type, strike, expiration, days_to_gain,
                                     number_of_contracts, average_cost_per_contract, risk_free_rate=0.05):
    try:
        try:
            strike = float(strike)
        except (TypeError, ValueError):
            return pd.DataFrame([{"Error": f"Invalid strike: {strike}"}])

        current_price = current_stock_price(ticker)
        if current_price <= 0:
            return pd.DataFrame([{"Error": f"Invalid or missing stock price for {ticker}"}])

        params = {
            "api_token": API_KEY,
            "filter[underlying_symbol]": ticker,
            "filter[type]": option_type,
            "filter[exp_date_from]": expiration,
            "filter[exp_date_to]": expiration,
            "filter[strike_from]": strike,
            "filter[strike_to]": strike,
            "page[limit]": 1
        }
        response = requests.get(BASE_URL, params=params)
        response.raise_for_status()
        data = response.json().get("data", [])

        if not data or "attributes" not in data[0]:
            return pd.DataFrame([{"Error": f"No matching option contract found for {ticker}"}])

        attr = data[0]["attributes"]
        last_price = float(attr.get("last", 0))

        try:
            iv = float(attr.get("volatility", 0.3))
            if not (0.01 <= iv <= 5):
                raise ValueError
        except (TypeError, ValueError):
            print(f"⚠️ Invalid IV for {ticker}. Using fallback 0.3")
            iv = 0.3

        exp_date = datetime.strptime(expiration, "%Y-%m-%d").replace(tzinfo=timezone.utc)
        today = datetime.now(timezone.utc)

        try:
            days_to_gain = int(days_to_gain)
        except (TypeError, ValueError):
            dte_days = (exp_date - today).days
            days_to_gain = max(1, int(dte_days * 0.5))

        eval_date = today + timedelta(days=days_to_gain)
        T_eval = max((exp_date - eval_date).days / 365, 0.05)

        try:
            number_of_contracts = int(number_of_contracts)
        except (TypeError, ValueError):
            number_of_contracts = 1

        try:
            average_cost_per_contract = float(average_cost_per_contract)
        except (TypeError, ValueError):
            average_cost_per_contract = 0.0

        if average_cost_per_contract <= 0:
            average_cost_per_contract = last_price

        total_cost = average_cost_per_contract * number_of_contracts * 100

        scenarios = [0.05, 0.10, 0.20, 0.50, 1.0, 2.0]
        rows = []

        for pct in scenarios:
            stock_up = current_price * (1 + pct)
            stock_down = max(current_price * (1 - pct), 0.01)  # Never allow 0 or negative

            if option_type == "call":
                premium_up = black_scholes_call_price(stock_up, strike, T_eval, risk_free_rate, iv)
                premium_down = black_scholes_call_price(stock_down, strike, T_eval, risk_free_rate, iv)
            else:
                premium_up = black_scholes_put_price(stock_up, strike, T_eval, risk_free_rate, iv)
                premium_down = black_scholes_put_price(stock_down, strike, T_eval, risk_free_rate, iv)

            rows.append({
                "Ticker": ticker,
                "Current Underlying": round(current_price, 2),
                "Option Type": option_type,
                "Strike": strike,
                "Expiration": expiration,
                "Number of Contracts": number_of_contracts,
                "Average Cost per Contract": round(average_cost_per_contract, 2),
                "Equity Invested": round(total_cost, 2),
                "Current Premium": round(last_price, 2),
                "Days to Gain": days_to_gain,
                "Scenario % Change": f"±{int(pct * 100)}%",
                "Simulated Underlying (+)": round(stock_up, 2),
                "Simulated Underlying (-)": round(stock_down, 2),
                "Simulated Premium (+)": round(premium_up, 2),
                "Simulated Premium (-)": round(premium_down, 2),
            })

        return pd.DataFrame(rows)

    except Exception as e:
        print(f"🚨 An error occurred during simulation: {e}")
        return pd.DataFrame([{"Error": str(e)}])

# --- Test Cases ---

print("--- Running Test Case 1: AAPL with all fields filled ---")
aapl_params = {
    "ticker": "AAPL",
    "option_type": "call",
    "strike": "250",
    "expiration": "2025-09-19",
    "days_to_gain": "34",
    "number_of_contracts": "1",
    "average_cost_per_contract": "0.75"
}
aapl_results = simulate_option_watchlist_single(**aapl_params)
display(aapl_results)

print("\n--- Running Test Case 2: GOOG with optional fields blank ---")
goog_params = {
    "ticker": "GOOG",
    "option_type": "call",
    "strike": "250",
    "expiration": "2027-01-15",
    "days_to_gain": "",  # Left blank
    "number_of_contracts": "",  # Left blank
    "average_cost_per_contract": ""  # Left blank
}
goog_results = simulate_option_watchlist_single(**goog_params)
display(goog_results)


--- Running Test Case 1: AAPL with all fields filled ---


,Ticker,Current Underlying,Option Type,Strike,Expiration,Number of Contracts,Average Cost per Contract,Equity Invested,Current Premium,Days to Gain,Scenario % Change,Simulated Underlying (+),Simulated Underlying (-),Simulated Premium (+),Simulated Premium (-)
0,AAPL,209.22,call,250.0,2025-09-19,1,0.75,75.0,0.75,34,±5%,219.68,198.76,0.33,0.01
1,AAPL,209.22,call,250.0,2025-09-19,1,0.75,75.0,0.75,34,±10%,230.14,188.30,1.32,0.00
2,AAPL,209.22,call,250.0,2025-09-19,1,0.75,75.0,0.75,34,±20%,251.06,167.38,8.51,0.00
3,AAPL,209.22,call,250.0,2025-09-19,1,0.75,75.0,0.75,34,±50%,313.83,104.61,64.93,0.00
4,AAPL,209.22,call,250.0,2025-09-19,1,0.75,75.0,0.75,34,±100%,418.44,0.01,169.53,0.00
5,AAPL,209.22,call,250.0,2025-09-19,1,0.75,75.0,0.75,34,±200%,627.66,0.01,378.75,0.00



--- Running Test Case 2: GOOG with optional fields blank ---


,Ticker,Current Underlying,Option Type,Strike,Expiration,Number of Contracts,Average Cost per Contract,Equity Invested,Current Premium,Days to Gain,Scenario % Change,Simulated Underlying (+),Simulated Underlying (-),Simulated Premium (+),Simulated Premium (-)
0,GOOG,182.33,call,250.0,2027-01-15,1,10.45,1045.0,10.45,274,±5%,191.45,173.21,5.87,2.65
1,GOOG,182.33,call,250.0,2027-01-15,1,10.45,1045.0,10.45,274,±10%,200.56,164.10,8.21,1.64
2,GOOG,182.33,call,250.0,2027-01-15,1,10.45,1045.0,10.45,274,±20%,218.80,145.86,14.52,0.52
3,GOOG,182.33,call,250.0,2027-01-15,1,10.45,1045.0,10.45,274,±50%,273.50,91.17,46.24,0.00
4,GOOG,182.33,call,250.0,2027-01-15,1,10.45,1045.0,10.45,274,±100%,364.66,0.01,125.75,0.00
5,GOOG,182.33,call,250.0,2027-01-15,1,10.45,1045.0,10.45,274,±200%,546.99,0.01,306.26,0.00
